In [78]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [79]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-01 21:06:10--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  4.79MB/s    in 0.2s    

2021-04-01 21:06:11 (4.79 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [81]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [83]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

### Create DataFrames to match tables

In [84]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
sports_df = df
sports_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [85]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = sports_df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)","customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35669025|             1|
|   48198949|            30|
|   43355824|             6|
|   33014261|             6|
|   23493243|             1|
|   30717305|             4|
|   15714077|             2|
|    7854719|             1|
|   12761428|             2|
|   14127895|             1|
|   51451778|             4|
|    3452965|             1|
|   40430762|             2|
|   27314089|            15|
|   11405991|             1|
|   20368048|             2|
|   33967841|             2|
|   10418855|             1|
|   43783459|             1|
|   50843047|             4|
+-----------+--------------+
only showing top 20 rows



In [91]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()
products_df = sports_df.select(["product_id","product_title"]).drop_duplicates(subset = ["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1570340439|Compass & Map Nav...|
|9879000633|Swordmaster - Uni...|
|B00004RAN1|24x7x7 Pro Cage Trap|
|B00005OU7P|Putt and Return P...|
|B00005V3EN|Airzone Trampolin...|
|B00008Z7M2|Koolatron 401619 ...|
|B0000AAEMO|Seahawks Franklin...|
|B0000AI0K1|Outdoor Cap Heavy...|
|B0000ANGL1|NFL Men's New Yor...|
|B0000ANHRR|Heatgear Loose Lo...|
|B0000AU3AK|Everlast Genuine ...|
|B0000AUTWD|Cubby 5011 Mini Mite|
|B0000AVZBW|G- Lox  Deerskin ...|
|B0000AXJT5|Pro Mex Professio...|
|B0000AXNWO|   Ande Fluorocarbon|
|B0000AXUET|Star brite Brush ...|
|B0000AY199|DU-BRO Fishing Tr...|
|B0000AYEE8|Attwood Stainiles...|
|B0000AYGZ1|Attwood Stainless...|
|B0000AZ8LS|Tacstar 4-Shot Re...|
+----------+--------------------+
only showing top 20 rows



# **Trouble-Shooting for Products_Table**

In [92]:
# Was originally Having issues loading the df to pgAdmin due to duplicates, drop_duplicates() alone wasn't working, need to include subset
#following script was used to test (1) the count of each product id and (2) which record was showing up as a duplicate in the data

prod_ct = products_df.groupby("product_id").agg({"product_id":"count"})
prod_ct.orderBy(prod_ct["count(product_id)"].desc()).show(10)

#originally had counts up to 400, now all records reflecting 1

+----------+-----------------+
|product_id|count(product_id)|
+----------+-----------------+
|B0085P7168|                1|
|B000LC65QA|                1|
|B00HA2ZKFQ|                1|
|B0027QSONK|                1|
|B006CUD8MY|                1|
|B000Y09Y0S|                1|
|B0058DQLO8|                1|
|B00XSP4E3I|                1|
|B00HS7GF70|                1|
|B000NONWIK|                1|
+----------+-----------------+
only showing top 10 rows



In [93]:
#originally showing 404 records - now only 1
products_df.filter(products_df["product_id"] == "B0082CXEI8").show(404)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0082CXEI8|Blue Sea Systems ...|
+----------+--------------------+



In [94]:
#originally showing 354 records - now only 1
products_df.filter(products_df["product_id"] == "B001U4ZZPK").show(354)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001U4ZZPK|Blue Sea Systems ...|
+----------+--------------------+



Troubleshoot End.
__________________________

**Challenge Continued:**


In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = sports_df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date",'yyyy-mm-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130| 2015-01-31|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679| 2015-01-31|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336| 2015-01-31|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227| 2015-01-31|
|R16PD71086BD2V|   29605511|B010T4IE2C|     787185588| 2015-01-31|
|R1Z8IFGWTRWXT6|   11112959|B004RKJGLS|      94127483| 2015-01-31|
|R3AUMSHAW73HWN|     108031|B005V3DCBU|     526977496| 2015-01-31|
|R2KWDWFOHGX6FL|   13981540|B00MHT9WN8|      26521265| 2015-01-31|
|R3H9543FWBWFBU|   37993909|B001CSIRQ8|     652431165| 2015-01-31|
| RUANXOQ9W3OU5|   26040213|B001KZ3NOO|     635861713| 2015-01-31|
|R31673RTGEZSW7|   34657602|B00005RCQS|      72099763| 2015-01-31|
|R22OQLFSH42RCM|   14346192|B00FA7RWVI|     757354022| 2015-01

In [101]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df=sports_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

In [103]:
# Troubleshoot: wouldn't load to pgAdmin
# ERROR: column "star_rating" is of type integer but expression is of type character varying  Hint: You will need to rewrite or cast the expression.
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [106]:
# cast a new datatype
new_vine_df = vine_df.withColumn("star_rating",vine_df.star_rating.cast('integer'))


### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [88]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<amazon_rds_endpoint>:5432/deliverable1"
config = {"user":"postgres", 
          "password": "<inserthere>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [95]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 9 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [107]:
# Write vine_df to table in RDS
# 25 minutes 24 s
new_vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)